In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
df = pd.read_csv('data/COVID19MEXICO2025.csv')

In [4]:
df["ID_REGISTRO"].duplicated().sum()

0

In [5]:
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,INDIGENA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_PCR,RESULTADO_PCR_COINFECCION,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL_COVID,CLASIFICACION_FINAL_FLU,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2025-03-04,0aa89f,1,10,9,2,15,9,3,1,2024-01-26,2024-01-25,9999-99-99,97,2,31,1,97,2,2,2,2,2,2,2,2,2,2,2,2,1,1,5,5,2,97,7,7,99,México,97,97
1,2025-03-04,g76bd10,1,4,32,2,32,32,56,2,2024-01-02,2024-01-02,9999-99-99,2,2,1,1,97,2,2,2,2,2,2,2,2,2,2,2,2,2,1,11,998,2,97,7,7,99,México,97,2
2,2025-03-04,gaa8ce3,1,12,9,1,9,15,13,1,2024-01-02,2024-01-01,9999-99-99,97,2,38,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,998,2,97,7,3,99,México,97,97
3,2025-03-04,g9637c9,1,4,25,1,25,25,12,2,2024-01-02,2024-01-01,9999-99-99,2,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,10,998,2,97,7,7,99,México,97,2
4,2025-03-04,g824984,1,4,32,2,32,32,17,2,2024-01-02,2024-01-02,9999-99-99,2,2,2,1,97,2,2,2,2,2,2,2,2,2,1,2,2,2,1,34,29,2,97,3,7,99,México,97,2


Se seleccionan unicamente los registros que sean confirmados y que sean casos positivos de COVID-19

Según el catalogo de datos, los valores 1,2,3 son casos confirmados 

In [6]:
df["CLASIFICACION_FINAL_COVID"].value_counts()

CLASIFICACION_FINAL_COVID
7    111695
6     84712
3     14648
5      4314
4       152
Name: count, dtype: int64

In [7]:
df = df[df["CLASIFICACION_FINAL_COVID"] == 3]

### Se crea una nueva variable que es saber si la persona falleció o no

In [8]:
df["DEFUNCION"] = np.where(df['FECHA_DEF'] == '9999-99-99', 2, 1)

### Se pasa a formato fecha

#### Primero se pasa a nulos los valores de 9999-99-99, que representan lo que no han fallecido

In [9]:
df['FECHA_DEF'] = df['FECHA_DEF'].replace('9999-99-99', np.nan)

In [10]:
df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'])

In [11]:
df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])

Vamos a calcular el numero de dias que han pasado desde la fecha de ingreso/sintomas hasta la fecha de defuncion

In [12]:
df['DIAS_SINTOMAS_DEF'] = (df['FECHA_DEF'] - df['FECHA_SINTOMAS']).dt.days
df['DIAS_SINTOMAS_ACT'] = (df['FECHA_ACTUALIZACION'] - df['FECHA_SINTOMAS']).dt.days

In [13]:
df[df['DEFUNCION']== 1][['DIAS_SINTOMAS_DEF','DIAS_SINTOMAS_ACT']]

,DIAS_SINTOMAS_DEF,DIAS_SINTOMAS_ACT
446,16.0,428
767,9.0,428
2203,9.0,426
2445,0.0,421
2514,4.0,423
...,...,...
202943,20.0,24
210125,17.0,27
210627,3.0,10
210631,5.0,10


In [14]:
df["FECHA_ACTUALIZACION"].describe()

count                  14648
mean     2025-03-04 00:00:00
min      2025-03-04 00:00:00
25%      2025-03-04 00:00:00
50%      2025-03-04 00:00:00
75%      2025-03-04 00:00:00
max      2025-03-04 00:00:00
Name: FECHA_ACTUALIZACION, dtype: object

Seleccionamos las columnas que nos interesan para el análisis

In [15]:
df_cox = df[["DEFUNCION","DIAS_SINTOMAS_ACT","DIAS_SINTOMAS_DEF", "NEUMONIA", "DIABETES", "EPOC", "ASMA", "INMUSUPR", "HIPERTENSION", "OTRA_COM", "CARDIOVASCULAR", "OBESIDAD", "RENAL_CRONICA", "TABAQUISMO"]]

In [16]:
df_cox.head()

,DEFUNCION,DIAS_SINTOMAS_ACT,DIAS_SINTOMAS_DEF,NEUMONIA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO
4,2,427,NaN,2,2,2,2,2,2,2,2,1,2,2
6,2,428,NaN,2,2,2,2,2,2,2,2,2,2,2
17,2,427,NaN,2,2,2,2,2,2,2,2,2,2,2
20,2,427,NaN,2,2,2,2,2,2,2,2,2,2,2
23,2,428,NaN,2,2,2,2,2,2,2,2,2,2,2


In [17]:
df_cox["DEFUNCION"].value_counts()

DEFUNCION
2    13774
1      874
Name: count, dtype: int64

Camabiar valores para que 1 sea si (prensenta condicion o afirmacion de nombre de columna) y 0 sea no (no presenta condicion o negación de nombre de columna)

In [18]:
df_cox.replace([1,2], [1,0], inplace=True)

C:\Users\daniel\AppData\Local\Temp\ipykernel_19416\3344868473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox.replace([1,2], [1,0], inplace=True)


Vamos a eliminar los registros con valores nulos

In [19]:
day_variables = df_cox[["DIAS_SINTOMAS_ACT","DIAS_SINTOMAS_DEF"]]

In [20]:
df_cox = df_cox.drop(day_variables, axis=1)

In [21]:
df_cox.replace([97,98,99], np.nan, inplace=True)

In [22]:
df_cox = df_cox.dropna()

In [23]:
df_cox["DEFUNCION"].value_counts()

DEFUNCION
0    13680
1      861
Name: count, dtype: int64

In [24]:
df_cox = df_cox.astype(int)

In [25]:
df_cox = df_cox.join(day_variables, how='left')

In [26]:
df_cox.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14541 entries, 4 to 214657
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DEFUNCION          14541 non-null  int32  
 1   NEUMONIA           14541 non-null  int32  
 2   DIABETES           14541 non-null  int32  
 3   EPOC               14541 non-null  int32  
 4   ASMA               14541 non-null  int32  
 5   INMUSUPR           14541 non-null  int32  
 6   HIPERTENSION       14541 non-null  int32  
 7   OTRA_COM           14541 non-null  int32  
 8   CARDIOVASCULAR     14541 non-null  int32  
 9   OBESIDAD           14541 non-null  int32  
 10  RENAL_CRONICA      14541 non-null  int32  
 11  TABAQUISMO         14541 non-null  int32  
 12  DIAS_SINTOMAS_ACT  14541 non-null  int64  
 13  DIAS_SINTOMAS_DEF  861 non-null    float64
dtypes: float64(1), int32(12), int64(1)
memory usage: 1.5 MB


In [27]:
df_cox.columns

Index(['DEFUNCION', 'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'DIAS_SINTOMAS_ACT',
       'DIAS_SINTOMAS_DEF'],
      dtype='object')

En los dias transcurridos hasta el evento, para los pacientes que fallecieron, los valores se reemplazan por los días desde la fecha del inicio de los sintomas, hasta la fecha de defuncion. Para los pacientes que no fallecieron, el valor de dias transcurridos se reemplazan por los dias desde el inicio de los sintomas hasta la ultima fecha de actualización del conjunto de datos(representa el ultimo dia de estudio)

In [28]:
df_cox["DIAS_TRANSCURRIDOS"] = np.where(df_cox['DEFUNCION'] == 0, df_cox["DIAS_SINTOMAS_ACT"], df_cox["DIAS_SINTOMAS_DEF"])

In [29]:
df_cox.drop(["DIAS_SINTOMAS_ACT", "DIAS_SINTOMAS_DEF"], axis=1, inplace=True)

In [30]:
df_cox.columns = ['DEFUNCION', 'NEUMONIA', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'DIAS_SINTOMAS_DEF']

In [31]:
df_cox = df_cox.astype(int)

In [33]:
df_cox.to_csv('data/df_cox2025.csv', index=False)